# Information Retrieval Baseline Code

## 대회 설명
Information Retrieval 경진대회에 오신 여러분 환영합니다! 🎉 <br>
본 대회에서는 과학 상식에 대한 멀터턴 질문에 대하여 RAG 기술을 활용하여 질문에 적합한 레퍼런스를 추출하고 이를 토대로 답변을 생성하는 어플리케이션을 구현해 봅니다. <br>
LLM은 GPT-3.5 버전을 이용하고, 레퍼런스 추출을 위한 검색엔진 구축은 Elasticsearch를 활용합니다.

## 데이터셋 준비
* 질문에 대한 레퍼런스로 활용될 수 있는 4272개의 과학 상식을 다루는 문서
 * Open Ko LLM Leaderboard에 들어가는 Ko-H4 데이터 중 MMLU, ARC 데이터를 기반으로 생성
 * Question과 answer를 paraphrase (question과 answer를 토대로 GPT-4 활용하여 document 생성)
 * 예시)
  ```
  수소, 산소, 질소 가스의 혼합물에서 평균 속도가 가장 빠른 분자는 수소입니다
. 수소 분자는 가장 가볍고 작은 원자로 구성되어 있기 때문에 다른 분자들보다 더 빠르게 움직>입니다. 이러한 이유로 수소 분자는 주어진 온도에서 가장 빠른 평균 속도를 가지고 있습니다. >수소 분자는 화학 반응에서도 활발하게 참여하며, 수소 연료로도 널리 사용됩니다. 따라서 수소 분자는 주어진 온도에서 평균 속도가 가장 빠른 분자입니다.
  ```

* 가상의 질문 220개
 * 과학 상식 질문 또는 일상 대화 메시지를 수동으로 생성
 * 220개 중 20개는 멀티턴 대화 형태의 질문, 20개는 일상 칫챗 대화
  * 예시)
  ```
{"eval_id": 0, "msg": [{"role": "user", "content": "나무의 분류에 대해 조사해 보기 위한 방
법은?"}], "refs": ["c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d"]}
{"eval_id": 1, "msg": [{"role": "user", "content": "각 나라에서의 공교육 지출 현황에 대해 알려줘."}], "refs": ["79c93deb-fe60-4c81-8d51-cb7400a0a156"]}
{"eval_id": 2, "msg": [{"role": "user", "content": "기억 상실증 걸리면 너무 무섭겠다."}, {"role": "assistant", "content": "네 맞습니다."}, {"role": "user", "content": "어떤 원인 때
문에 발생하는지 궁금해."}], "refs": ["25de4ffd-cee4-4f27-907e-fd6b802c6ede"]}
  ```

In [1]:
 # 색인 대상 문서 및 평가 데이터 다운로드
 !wget https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000291/data/data.tar.gz
 !tar -xzvf data.tar.gz

--2024-02-14 05:15:10--  https://aistages-api-public-prod.s3.amazonaws.com/app/Competitions/000291/data/data.tar.gz
Resolving aistages-api-public-prod.s3.amazonaws.com (aistages-api-public-prod.s3.amazonaws.com)... 52.219.206.95, 52.219.202.27, 52.219.56.70, ...
Connecting to aistages-api-public-prod.s3.amazonaws.com (aistages-api-public-prod.s3.amazonaws.com)|52.219.206.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1068264 (1.0M) [binary/octet-stream]
Saving to: ‘data.tar.gz’

data.tar.gz         100%[===================>]   1.02M  1.49MB/s    in 0.7s    

2024-02-14 05:15:12 (1.49 MB/s) - ‘data.tar.gz’ saved [1068264/1068264]

data/
data/eval.jsonl
data/documents.jsonl


## 환경 설정
검색엔진을 위한 Elasitcsearch, 임베딩 생성을 위한 sentence transformers, LLM 사용을 위한 openai client 설치가 필요합니다.

In [1]:
# Elasticsearch python 패키지 설치
!pip install elasticsearch==8.8.0

# OpenAI Python 패키지 설치
!pip install openai==1.7.2

# 임베딩 생성을 위한 벡터 인코더 설치
!pip install sentence-transformers==2.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 4.4 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 5.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:0000:0100:01m
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125937 sha256=a6b7e361d9a6d35feee368a7559085ee84b2d0243431cf89f0e983e370814e04
  Stored in directory: /home/demyank/.cache/pip/wheels/ff/27/bf/ffba8b318b02d7f691a57084ee154e26ed24d012b0c780

## 검색엔진 준비 - Elasticsearch




In [3]:
# 데몬이 구동되었는지 확인 (세개의 daemon process가 있어야 함)
!ps -ef | grep elasticsearch

demyank  69360 68765  0 11:59 pts/11   00:00:00 /usr/bin/zsh -c ps -ef | grep elasticsearch
demyank  69362 69360  0 11:59 pts/11   00:00:00 grep elasticsearch


In [17]:
import os
import torch
from elasticsearch import Elasticsearch, helpers
from peft import LoraConfig, AutoPeftModelForCausalLM, PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments, pipeline, DataCollatorForSeq2Seq
from datetime import datetime

In [2]:
# Elasticsearch credentials (adjust these as per your Elasticsearch setup)
es_username = 'elastic'
es_password = os.getenv('ES_PASSWORD')  # Replace 'your_password_here' with your actual Elasticsearch password

# Path to your CA certificate on your host (adjust this after copying it from the container)
ca_cert_path = os.getenv('HTTP_CA_CRT_PATH')  # Update this to the actual path where you store your copied CA certificate

# Create an Elasticsearch client
es = Elasticsearch(
    ['https://localhost:9200'],
    basic_auth=(es_username, es_password),
    ca_certs=ca_cert_path
)

# Print Elasticsearch client information to verify connectivity
print(es.info())


{'name': '8b4375334f9c', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'XuaZu5E-QgWjjKSFBuUCaQ', 'version': {'number': '8.13.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '16cc90cd2d08a3147ce02b07e50894bc060a4cbf', 'build_date': '2024-04-05T14:45:26.420424304Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [3]:
from sentence_transformers import SentenceTransformer

# checkpoint_path = 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'

# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
model = SentenceTransformer('output/scientific_continue_training_kr_sbert_v40k_klueNLI_augSTS')

In [4]:
# SetntenceTransformer를 이용하여 임베딩 생성
def get_embedding(sentences):
    # return model.encode(sentences)
    return [model.encode(sentence) for sentence in sentences]


# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = get_embedding(contents)
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings


# 새로운 index 생성
def create_es_index(index, settings, mappings):
    # 인덱스가 이미 존재하는지 확인
    if es.indices.exists(index=index):
        # 인덱스가 이미 존재하면 설정을 새로운 것으로 갱신하기 위해 삭제
        es.indices.delete(index=index)
    # 지정된 설정으로 새로운 인덱스 생성
    es.indices.create(index=index, settings=settings, mappings=mappings)


# 지정된 인덱스 삭제
def delete_es_index(index):
    es.indices.delete(index=index)


# Elasticsearch 헬퍼 함수를 사용하여 대량 인덱싱 수행
def bulk_add(index, docs):
    # 대량 인덱싱 작업을 준비
    actions = [
        {
            '_index': index,
            '_source': doc
        }
        for doc in docs
    ]
    return helpers.bulk(es, actions)


# 역색인을 이용한 검색
def sparse_retrieve(query_str, size):
    query = {
        "match": {
            "content": {
                "query": query_str
            }
        }
    }
    return es.search(index="test", query=query, size=size, sort="_score")


# Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_embedding([query_str])[0]

    # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정
    knn = {
        "field": "embeddings",
        "query_vector": query_embedding.tolist(),
        "k": size,
        "num_candidates": 100
    }

    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index="test", knn=knn)


In [5]:
# 색인을 위한 setting 설정
settings = {
    "analysis": {
        "analyzer": {
            "nori": {
                "type": "custom",
                "tokenizer": "nori_tokenizer",
                "decompound_mode": "mixed",
                "filter": ["nori_posfilter"]
            }
        },
        "filter": {
            "nori_posfilter": {
                "type": "nori_part_of_speech",
                # 어미, 조사, 구분자, 줄임표, 지정사, 보조 용언 등
                "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]
            }
        }
    }
}

# 색인을 위한 mapping 설정 (역색인 필드, 임베딩 필드 모두 설정)
mappings = {
    "properties": {
        "content": {"type": "text", "analyzer": "nori"},
        "embeddings": {
            "type": "dense_vector",
            "dims": 768,
            "index": True,
            "similarity": "l2_norm"
        }
    }
}


In [6]:
# settings, mappings 설정된 내용으로 'test' 인덱스 생성
create_es_index("test", settings, mappings)

In [7]:
import json

# 문서의 content 필드에 대한 임베딩 생성
index_docs = []
with open("data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]
embeddings = get_embeddings_in_batches(docs)

# 생성한 임베딩을 색인할 필드로 추가
for doc, embedding in zip(docs, embeddings):
    # doc["embeddings"] = embedding.tolist()
    doc["embeddings"] = embedding
    index_docs.append(doc)

# 'test' 인덱스에 대량 문서 추가
ret = bulk_add("test", index_docs)

# 색인이 잘 되었는지 확인 (색인된 총 문서수가 출력되어야 함)
print(ret)


batch 0
batch 100
batch 200
batch 300
batch 400
batch 500
batch 600
batch 700
batch 800
batch 900
batch 1000
batch 1100
batch 1200
batch 1300
batch 1400
batch 1500
batch 1600
batch 1700
batch 1800
batch 1900
batch 2000
batch 2100
batch 2200
batch 2300
batch 2400
batch 2500
batch 2600
batch 2700
batch 2800
batch 2900
batch 3000
batch 3100
batch 3200
batch 3300
batch 3400
batch 3500
batch 3600
batch 3700
batch 3800
batch 3900
batch 4000
batch 4100
batch 4200
(4272, [])


In [8]:
# 검색엔진에 색인이 잘 되었는지 테스트하기 위한 질의
test_query = "금성이 다른 행성들보다 밝게 보이는 이유는 무엇인가요?"

In [9]:
# 역색인을 사용하는 검색 예제
search_result_retrieve = sparse_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 33.68643 source: 금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.
score: 18.915426 source: 금성은 태양계의 두 번째로 가까운 행성입니다. 이 행성의 대략적인 나이는 7억 5천만 년으로 추정됩니다. 금성은 지구와 매우 비슷한 크기와 구성을 가지고 있으며, 약 90% 이상이 이산화탄소로 이루어져 있습니다. 이 행성은 매우 뜨거운 온도와 압력을 가지고 있어서 인간이 살 수 있는 환경이 아닙니다. 금성의 대기는 두꺼워서 태양의 열을 가두고 있어서 행성의 표면은 평균 온도가 약 450도로 매우 뜨거운 상태입니다. 또한, 금성은 자전 속도가 매우 빠르기 때문에 하루가 지구의 약 243일과 같습니다. 이러한 특징들로 인해 금성은 우리 태양계에서 가장 가혹한 환경을 가진 행성 중 하나로 알려져 있습니다.
score: 18.728905 source: 메릴랜드 Space Grant Observatory는 볼티모어에 위치해 있습니다. 학생들은 이 망원경을 방문하여 별, 행성, 달에 대해 배웠습니다. 그들은 아래와 같은 정보를 기록했습니다. 첫째, 별 패턴은 그대로 유지되지만, 하늘에서의 위치는 변하는 것처럼 보입니다. 둘째, 태양, 행성, 달은 하늘에서 움직이는 것처럼 보입니다. 셋째, 켄타우루스자리의 프록시마 성은 우리 태양계에서 가장 가까운 별입니다. 넷째, 북극성은 소 북두칠성이라 불리는 별 패턴의 일부입니다. 그렇다면 태양이 매일 하늘을 가로질러 움직이는 것처럼 보이는 이유는 무엇일까요? 지구가 자전축을 중심으로 회전하기

In [10]:
# Vector 유사도 사용한 검색 예제
search_result_retrieve = dense_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 0.012061637 source: 금성은 태양계에서 가장 가까운 행성 중 하나입니다. 그러나 화성이나 지구처럼 계절이 없는 이유는 금성의 자전축이 태양계의 평면에 거의 수직이기 때문입니다. 자전축이 수직이기 때문에 금성은 태양으로부터 받는 햇빛의 양이 일정하게 유지됩니다. 이로 인해 금성은 계절 변화가 없으며 항상 일정한 온도를 유지합니다. 이러한 환경은 생명체에게는 적합하지 않을 수 있지만, 금성의 특이한 기후 조건은 우주 탐사에 대한 연구에 많은 도움을 주고 있습니다. 금성은 여전히 우리에게 알려지지 않은 많은 비밀을 품고 있으며, 미래에 더 많은 연구와 탐사가 이루어질 것으로 기대됩니다.
score: 0.011799942 source: 금성은 태양계의 두 번째로 가까운 행성입니다. 이 행성의 대략적인 나이는 7억 5천만 년으로 추정됩니다. 금성은 지구와 매우 비슷한 크기와 구성을 가지고 있으며, 약 90% 이상이 이산화탄소로 이루어져 있습니다. 이 행성은 매우 뜨거운 온도와 압력을 가지고 있어서 인간이 살 수 있는 환경이 아닙니다. 금성의 대기는 두꺼워서 태양의 열을 가두고 있어서 행성의 표면은 평균 온도가 약 450도로 매우 뜨거운 상태입니다. 또한, 금성은 자전 속도가 매우 빠르기 때문에 하루가 지구의 약 243일과 같습니다. 이러한 특징들로 인해 금성은 우리 태양계에서 가장 가혹한 환경을 가진 행성 중 하나로 알려져 있습니다.
score: 0.005973127 source: 금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.


## RAG 구현

준비된 검색엔진과 LLM을 활용하셔 대화형 RAG 구현

In [29]:
from openai import OpenAI
import traceback

# OpenAI API 키를 환경변수에 설정
# os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
#os.environ["OPENAI_API_KEY"] = "Your API Key"

client = OpenAI(organization=os.getenv('ORGANIZATION_ID'))
# 사용할 모델을 설정(여기서는 gpt-3.5-turbo-1106 모델 사용)
llm_model = "gpt-4-turbo" # "gpt-3.5-turbo" # 
# llm_model = "gpt-3.5-turbo"


In [30]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
persona_qa = """
## Role: 과학 지식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
- 결과는 json 형태로 생성하고, 반드시 'is_science' 필드를 추가하여 질문이 과학 지식에 관련된 내용이면 true를, 과학 지식에 관련된 내용이 아니라면 false를 value로 만든다. 답변 필드의 이름은 'answer'로 통일한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 과학 지식 전문가

## Instruction
- 제대로 답변을 하지 않으면 기업이 엄청난 손해를 보게된다.
- 사용자가 대화를 통해 생물학, 물리학, 지구과학, 화학, 의학, 공학, 컴퓨터, 농업, 산업공학 등 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
- 사회, 경제, 시사, 문화, 문학, 역사, 일상, 생활, 사교 등 과학 지식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
- 결과는 json 형태로 생성하고, 반드시 'is_science' 필드를 추가하여 질문이 과학 지식에 관련된 내용이면 true를, 과학 지식에 관련된 내용이 아니라면 false를 value로 만든다. 답변 필드의 이름은 'answer'로 통일한다.
"""

# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search relevant documents",
            "parameters": {
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "Final query suitable for use in search from the user messages history."
                    }
                },
                "required": ["standalone_query"],
                "type": "object"
            }
        }
    },
]


In [31]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        ### 이 부분을 바꿔서 json으로 만들어 줄까...
        result = client.chat.completions.create(
            model=llm_model,
            response_format={"type": "json_object"},
            messages=msg,
            tools=tools,
            #tool_choice={"type": "function", "function": {"name": "search"}},
            temperature=0,
            seed=1,
            timeout=60
        )
    except Exception as e:
        traceback.print_exc()
        return response

    # 검색이 필요한 경우 검색 호출후 결과를 활용하여 답변 생성
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        standalone_query = function_args.get("standalone_query")

        # Baseline으로는 sparse_retrieve만 사용하여 검색 결과 추출
        search_result = dense_retrieve(standalone_query, 3)

        response["standalone_query"] = standalone_query
        retrieved_context = []
        for i,rst in enumerate(search_result['hits']['hits']):
            retrieved_context.append(rst["_source"]["content"])
            response["topk"].append(rst["_source"]["docid"])
            response["references"].append({"score": rst["_score"], "content": rst["_source"]["content"]})

        content = json.dumps(retrieved_context)
        messages.append({"role": "assistant", "content": content})
        msg = [{"role": "system", "content": persona_qa}] + messages
        try:
            qaresult = client.chat.completions.create(
                    model=llm_model,
                    response_format={"type": "json_object"},
                    messages=msg,
                    temperature=0,
                    seed=1,
                    timeout=30
                )
        except Exception as e:
            traceback.print_exc()
            return response
        response["answer"] = qaresult.choices[0].message.content

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content

    return response



In [32]:
import json

# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출 후 파일에 저장
def eval_rag(eval_filename, output_filename, start_idx=-1):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx=0
        for line in f:
            if idx < start_idx:
                idx += 1
                continue
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            # output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}

            # json.loads(response)
            res_answer = json.loads(response["answer"])
            is_science = res_answer.get("is_science", False)
            if not is_science:
                answer = "과학 지식에 관련한 질문이 아닙니다."
                topk = []
            else:
                answer = res_answer['answer']
                topk = response["topk"]
            
            # json format으로 생성했을 때.
            output = {"eval_id": j["eval_id"], 
                      "standalone_query": response["standalone_query"], 
                      "topk": topk, 
                      "answer": answer, 
                      "references": response["references"]}
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1


In [26]:
# 역색인을 사용하는 검색 예제
q1 = '나무의 분류에 대해 조사해 보기 위한 방법은?'
search_result_retrieve = sparse_retrieve(q1, 3)
search_result_retrieve

ObjectApiResponse({'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 4238, 'relation': 'eq'}, 'max_score': 20.857122, 'hits': [{'_index': 'test', '_id': 'cZphOI8B-sAIPHWjBLOI', '_score': 20.857122, '_source': {'docid': 'c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d', 'src': 'ko_ai2_arc__ARC_Challenge__test', 'content': '한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다. 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다. 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다.', 'embeddings': [0.17039655148983002, -0.13365022838115692, -0.20746152102947235, 0.9684341549873352, 0.09740294516086578, 0.3687126636505127, 1.2090917825698853, -0.09777592867612839, -0.12069620937108994, -0.4030536413192749, -0.835671603679657, 0.40196850895881653, -0.2718091607093811, 1.1951520442962

In [23]:

# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("data/eval.jsonl", f"submission_{datetime.now().strftime('%Y-%m-%d_%H-%M')}.csv")

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]


KeyboardInterrupt: 

In [33]:
eval_rag("data/eval.jsonl", f"submission_{datetime.now().strftime('%Y-%m-%d_%H-%M')}.csv", 99)

Test 99
Question: [{'role': 'user', 'content': '너 잘하는게 뭐야?'}]
Answer: {
  "answer": "저는 과학 지식에 관한 질문에 답변하는 것을 잘합니다. 과학적인 질문이 있으시면 언제든지 도와드릴 수 있어요!",
  "is_science": false
}

Test 100
Question: [{'role': 'user', 'content': '지구에서 새로운 땅이 생겨나는 메커니즘은?'}]
Answer: {
  "answer": "지구에서 새로운 땅이 생겨나는 주요 메커니즘은 판구조론에 의해 설명됩니다. 지구의 리소스피어는 여러 개의 판으로 나뉘어 있으며, 이 판들이 움직이면서 상호작용합니다. 판들이 서로 멀어지는 곳에서는 맨틀에서 물질이 상승하여 새로운 지각을 형성하는 해령이 발생하고, 판이 서로 부딪히거나 한 판이 다른 판 아래로 들어가는 곳에서는 산맥이 형성되거나 화산 활동이 일어나 새로운 땅이 생성됩니다.",
  "is_science": true
}

Test 101
Question: [{'role': 'user', 'content': '더러워진 엔진오일을 잘 처리하는 방법에 대해 알려줘.'}]
Answer: {
  "answer": "더러워진 엔진오일은 적절하게 처리하는 것이 중요합니다. 사용한 엔진오일은 재활용 센터나 정비소에 가져가서 제대로 처리하도록 합니다. 이러한 곳들은 오일을 안전하게 수거하여 재활용하거나 적절히 처리할 수 있는 시설을 갖추고 있습니다. 절대로 일반 쓰레기와 함께 버리거나 환경에 방출해서는 안 됩니다.",
  "is_science": true
}

Test 102
Question: [{'role': 'user', 'content': '너 정말 똑똑하다!'}]
Answer: {
  "answer": "감사합니다! 무엇을 도와드릴까요?",
  "is_science": false
}

Test 103
Question: [{'role': 'user', 'content': '다람쥐가

In [20]:
!wc -l sample_submission.csv

6 sample_submission.csv


#Reference

## Required Package

openai==1.7.2 <br>
elasticsearch==8.8.0 <br>
sentence_transformers==2.2.2 <br>



## 콘텐츠 라이선스

저작권 : <font color='blue'> <b> ©2023 by Upstage X fastcampus Co., Ltd. All rights reserved.</font></b>

<font color='red'><b>WARNING</font> : 본 교육 콘텐츠의 지식재산권은 업스테이지 및 패스트캠퍼스에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다. </b>